In [1]:
import matplotlib.pyplot as plt
import pandas as pd
import seaborn
import glob
import re

In [3]:
RoseTTA_Fold_scorefile = 'your_filename_here.csv'
scored_seqs = pd.read_csv(RoseTTA_Fold_scorefile)

In [4]:
def glob_fasta_into_dictionary(glob_string):
    fasta_files = glob.glob(glob_string)
    fasta_dictionary = {}
    
    for file in fasta_files:
        with open(file, 'r') as input_file:
            fasta_dictionary[re.sub(r'.*/(.*)\.fa.*',r'\1',file)] = input_file.readlines()[1]
    
    return fasta_dictionary


In [6]:
seq_dict = glob_fasta_into_dictionary('example/initial_seq/*fas')
seq_list = []
mutation_list = []

for name in scored_seqs['name']:
    seq_list.append(seq_dict[name])
    mutation_list.append(name.count('MUT'))
    
scored_seqs['seq'] = seq_list
scored_seqs['mut'] = mutation_list
scored_seqs



In [7]:
plt.rcParams["figure.figsize"] = (10,10)
seaborn.scatterplot(data=scored_seqs, x='kl', y='cce', hue='mut', s=6, palette='rocket') #, hue='mut', s=6,  palette='Spectral')
plt.autoscale()
plt.tight_layout()

In [9]:
base_pattern = []
for name in scored_seqs['name']:
    base_pattern.append(re.sub(r'(.*)_[A-Z]+', r'\1', name))

scored_seqs['base'] = base_pattern


In [12]:
top_sequences_from_each_pattern = []

for base in scored_seqs['base'].unique():
    base_scored_seqs = scored_seqs[scored_seqs['base'] == base]
#     print (base_scored_seqs.quantile(.90))
    max_index = base_scored_seqs['kl'].idxmax()
    #print (max_index, base_scored_seqs['kl'][max_index])
    top_percentile = base_scored_seqs[base_scored_seqs['kl'] > base_scored_seqs.quantile(.95)['kl']]
    top_percentile = top_percentile[top_percentile['cce'] > base_scored_seqs.quantile(.95)['cce']]
#     print (len(base_scored_seqs), len(top_percentile))
    if len(top_percentile) > 5:
        top_percentile = top_percentile.sort_values('kl', ascending = False).head(5)
#     print (top_percentile)
#     print (len(top_percentile), len(top_percentile))
    for base, name, seq, kl in zip(top_percentile['base'], top_percentile['name'], top_percentile['seq'], top_percentile['kl']):
        top_sequences_from_each_pattern.append((base, name, seq, kl))
#     print ()
    
# print (len(scored_seqs), len(best_for_each_pattern))
print (len(top_sequences_from_each_pattern))
# best_for_each_pattern

bases = [base for base, name, seq, kl in top_sequences_from_each_pattern]
print (len(set(bases)))
# top5_names = [name for base, name, seq, kl in top_sequences_from_each_pattern]

with open('./example/initial_seq/selected.fas', 'w') as output:
    for base, name, seq, kl in top_sequences_from_each_pattern:
        print ('>{0}_{2}\n{1}'.format(name, seq.strip(), str(round(kl,2)).replace('.','_')), file=output )
#         print (kl)

233
50
